In [1]:
import pandas as pd

In [2]:
df1_5 = pd.read_csv("./result1-5.csv", encoding='utf-8')
df1_5.set_index('uid', inplace=True)

df6_10 = pd.read_csv("./result6-10.csv",encoding='utf-8')
df6_10.set_index('uid', inplace=True)

df11_15 = pd.read_csv("./result11-15.csv",encoding='utf-8')
df11_15.set_index('uid', inplace=True)

df16_other = pd.read_csv("./result16-other.csv",encoding='utf-8')
df16_other.set_index('uid', inplace=True)

In [3]:
data1 = pd.merge(df1_5,df6_10,how='left',left_index=True, right_index = True,sort=False,copy=True)
data1 = pd.merge(data1, df11_15,how='left',left_index=True, right_index = True,sort=False,copy=True)
data1 = pd.merge(data1, df16_other,how='left',left_index=True, right_index = True,sort=False,copy=True)

In [60]:
data1.to_csv('./result.csv',encoding='utf-8')

In [4]:
from clickhouse_driver import Client

host = "localhost"
port = 9099
user = "sjjc"
password = "taosan506"
database = "dm"
send_receive_timeout = 10
client = Client(host=host, port=port, user=user, password=password, database=database,
                send_receive_timeout=send_receive_timeout)
star_df_list_sql = """
select uid, star_level from dm.pri_star_info;
"""
star_df_list = client.execute(star_df_list_sql)
attr_list = ['uid', 'star_level']
star_df = pd.DataFrame(columns=attr_list, data=star_df_list)
star_df.set_index('uid', inplace=True)

In [5]:
data_all = pd.merge(star_df,data1,how='left',left_index=True, right_index = True,sort=False,copy=True)

In [63]:
from sklearn.model_selection import train_test_split
known_data = data_all[(data_all['star_level']!='-1')]

In [64]:
cols = [i for i in known_data.columns]

In [65]:
x = known_data.iloc[:,1:]
x.fillna(0,inplace=True)
from  sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()  #实例化
scaler=scaler.fit(x)  #fit,在这里本质就是生成min(x)和max(x)
x=scaler.transform(x)   #通过接口导出结果

In [73]:
y = known_data.iloc[:,0:1]['star_level'].tolist()

In [74]:
x_train, x_test, y_train,y_test = train_test_split(x, y, test_size=0.25)

/home/fguohao/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fguohao/.local/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearSVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


ValueError: X has 1 features, but LinearSVC is expecting 111 features as input.

       1     2     3  4    5   6  7  8  9
1  31746   412    90  1    3   0  0  0  0
2   9725  1095   172  0    3   1  0  0  0
3   6555  1154  1849  1   12   2  0  0  0
4    797   168  1515  1   27  13  0  0  0
5    229    44  1295  0  150  47  0  0  0
6     19    10   564  0  266  57  0  0  0
7      0     0     1  0    5   5  0  0  0
8      0     0     0  0    0   1  0  0  0
9      0     0     0  0    0   3  0  0  0


In [45]:
print("precision:" , precision_score(y_test, result))
print("recall:" , recall_score(y_test, result))

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [75]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(multi_class='multinomial',solver='newton-cg')

In [76]:
log_reg.fit(x_train, y_train)
predict_train = log_reg.predict(x_train)

In [77]:
from sklearn.metrics import r2_score
print("train precision:" , r2_score(y_train, predict_train))
predict_test = log_reg.predict(x_test)
print("r2 precision:" , r2_score(y_test, predict_test))

train precision: 0.016802156288668435
r2 precision: 0.013981988969606673


/home/fguohao/.local/lib/python3.7/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/home/fguohao/.local/lib/python3.7/site-packages/sklearn/metrics/_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
/home/fguohao/.local/lib/python3.7/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and wil

In [78]:
def true_score(result, test):
    t = 0
    for i in range(len(result)):
        if result[i]==test[i]:
            t = t+1
    return t/len(result)

In [80]:
y_test = y_test['star_level'].tolist()
print(true_score(predict_train, y_train))
print(true_score(predict_test,y_test))

0.5874585752766913
0.5886315862021434


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, precision_score, recall_score
modelsvc = SVC()
modelsvc.fit(x_train, y_train)

In [ ]:
result = modelsvc.predict(x_test)
cm = pd.DataFrame(
    confusion_matrix(y_test, result),
    index = ['1','2','3','4','5','6','7','8','9'],
    columns = ['1','2','3','4','5','6','7','8','9'],
)
print(cm)

In [6]:
data_all['star_level'] = data_all['star_level'].apply(int)

In [10]:
correlations = data_all.corr()['star_level']

In [15]:
correlations.T

star_level            1.000000
djk_num_x             0.027458
normal_djk_num_x      0.016622
abnormal_djk_num_x   -0.020110
not_using_djk         0.019519
                        ...   
base_delay_bal       -0.010216
sex                   0.004387
birthday              0.159269
education            -0.011313
marrige               0.134845
Name: star_level, Length: 124, dtype: float64

In [20]:
attrs = ['base_max_actu_intr',
'base_max_frst_intr',
'dsf_mx_count',
'max_intr',
'base_count_loan_amt',
'abnormal_djk_num_y',
'abnormal_djk_num_x',
'max_pay_term',
'owed_int_times_90g',
'month_avg_huanx_count',
'overdue_times_90g',
'owed_int_times_90l',
'overdue_times_90l',
'base_bad_bal',
'djk_sum_dlay_mths',
'education',
'base_due_intr',
'base_delay_bal',
'contract_mx_dull_bal',
         'contract_buy_house_time',
'djk_is_etc',
'dsf_mx_sum',
'base_max_book_acct_amt',
'tran_amt_huanb',
'month_avg_shop_count',
'djk_credit',
'has_car_etc',
'sjyh_tran_amt_count',
'tran_amt_duebill',
'normal_djk_num_y',
'normal_djk_num_x',
'djk_mode_online',
'dsf_mx_avg',
'contract_consume_sum',
'dsf_mx_max',
'month_avg_sdrq_amt',
'contract_mx_num',
'not_using_djk',
'djk_bankacct_num',
'contract_mx_buss_amt',
'offline_tran_duebill',
'month_avg_sbyb_amt',
'djk_num_x',
'djk_num_y',
'contract_business_sum',
'contract_business',
'contract_business_term_month',
'djk_bankacct_sum',
'sjyh_tran_amt_sum',
'contract_consume_term_month',
'cust_bal_sum',
'contract_consume_time',
'cust_fin_bal',
'contract_consume',
'month_avg_sbyb_count',
'tran_amt_gzdf',
'cust_cd_bal',
'month_avg_sdrq_count',
'marrige',
'tran_times_gzdf',
'cust_sa_bal',
'birthday',
'has_social_security_gzdf',
'cust_all_bal_sum',
 'star_level'
         ]
x_feats = data_all.loc[:,attrs]

In [21]:
x_feats.fillna(0,inplace=True)

In [24]:
from sklearn.model_selection import train_test_split
known_data = x_feats[(x_feats['star_level']!='-1')]

In [31]:
x = known_data.iloc[:,0:64]
y = known_data.iloc[:,64:]
y = y['star_level'].tolist()

In [32]:
x_train, x_test, y_train,y_test = train_test_split(x, y, test_size=0.25)

In [33]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(multi_class='multinomial',solver='newton-cg')

In [35]:
log_reg.fit(x_train, y_train)

KeyboardInterrupt: 

In [ ]:
predict_train = log_reg.predict(x_train)

In [36]:
def true_score(result, test):
    t = 0
    diff = 0
    for i in range(len(result)):
        if result[i]==test[i]:
            t = t+1
        else:
            diff = diff + abs(int(result[i])-int(test[i]))

    print(diff/len(result))
    return t/len(result)

In [ ]:
print(true_score(predict_train, y_train))
print(true_score(predict_test,y_test))

In [37]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=750
,n_jobs=-1,max_leaf_nodes=66)
rf_clf.fit(x_train,y_train)

RandomForestClassifier(max_leaf_nodes=66, n_estimators=750, n_jobs=-1)

In [38]:
y_pred = rf_clf.predict(x_test)
print(true_score(y_test, y_pred))

0.7784077616459093
0.6304685887290993


In [40]:
from sklearn.metrics import confusion_matrix
cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index = ['-1','1','2','3','4','5','6','7','8','9'],
    columns = ['-1','1','2','3','4','5','6','7','8','9']
)
print(cm)

     -1      1     2     3  4   5    6  7  8  9
-1   95   8513  2319  3445  0  25   45  0  0  0
1    17  30490  1586   275  0   0    0  0  0  0
2    11   3110  6982   973  0   5    0  0  0  0
3    26    586   858  8086  0   8    1  0  0  0
4    71     60    74  2299  0  13    3  0  0  0
5   155     34    32  1521  0  41   29  0  0  0
6    90      9    11   612  0  15  119  0  0  0
7     1      0     0     8  0   0    6  0  0  0
8     2      0     0     1  0   0    1  0  0  0
9     0      0     0     0  0   0    2  0  0  0
